# Mortgage Workflow with Deep Learning - Basic (No Ignite)

Original workflow by Even Oldridge - Modified by Rick Zamora

### Dataset

The dataset used with this workflow is derived from [Fannie Mae’s Single-Family Loan Performance Data](http://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html) with all rights reserved by Fannie Mae. This processed dataset is redistributed with permission and consent from Fannie Mae.

In [2]:
using_docker = False
if using_docker:
    data_dir = '/data/mortgage/'
    !ls -al --block-size=M /data/mortgage/
else:
    data_dir = '/datasets/mortgage/post_etl/dnn/'
    !ls -al --block-size=M /datasets/mortgage/post_etl/dnn/

total 0M
drwxr-xr-x 1 eoldridge nvidia 0M May 29 11:38 .
drwxr-xr-x 1 eoldridge nvidia 0M May 29 11:38 ..
drwxr-xr-x 1 eoldridge nvidia 0M May 29 11:38 test
drwxr-xr-x 1 eoldridge nvidia 0M May 29 11:38 train
drwxr-xr-x 1 eoldridge nvidia 0M May 29 11:38 validation


### Run parallel training script and collect profile

In [4]:
import os

nranks = 4
os.system('mpirun -n '+str(nranks)+' python -m cProfile -o program.prof parallel_training.py --batched --adam --epochs 3 --hvd --fp16-allreduce')

0